In [1]:
# import des librairies dont nous aurons besoin
import pandas as pd
import pprint
import numpy as np
import re
import missingno as msno 
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sms
from sklearn.linear_model import ElasticNet, Lasso, LinearRegression, Ridge
from sklearn.datasets import make_regression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder, RobustScaler, LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector, ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, cross_validate

# Import des données

In [2]:
df_clean = pd.read_csv("df_clean.csv")
df_clean

,State,NAICS,Term,FranchiseCode,UrbanRural,GrAppv,Default
0,IN,45,84,1,0,6000000,0
1,IN,72,60,1,0,4000000,0
2,IN,62,180,1,0,28700000,0
3,OK,0,60,1,0,3500000,0
4,FL,0,240,1,0,22900000,0
...,...,...,...,...,...,...,...
897149,OH,45,60,1,0,7000000,0
897150,OH,45,60,1,0,8500000,0
897151,CA,33,108,1,0,30000000,0
897152,HI,0,60,1,0,7500000,1


# PIPELINE

In [3]:
X = df_clean.drop('Default', axis=1)
y = df_clean['Default']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
numeric_features = ["Term", "FranchiseCode", "UrbanRural", "GrAppv"]
categorial_features = [ "State", "NAICS"]

In [4]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder
scale_one_hot = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(), categorial_features),
    ('standard_scaler', StandardScaler(), numeric_features)] )

## Modélisation

In [6]:
pipe = make_pipeline(scale_one_hot,RandomForestClassifier())

In [7]:
pipe.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
# grid.best_estimator()

In [10]:
# Optimisation des hyperparamètres
from sklearn.model_selection import GridSearchCV
param_grid = { 
    'randomforestclassifier__n_estimators': [200, 300],
    'randomforestclassifier__max_features': ['auto', 'sqrt', 'log2'],
    'randomforestclassifier__max_depth' : [4,5],
}

CV = GridSearchCV(pipe, param_grid, verbose=2, n_jobs=-1)
CV.fit(X_train, y_train)
print(CV.best_params_)

# Réduction de la dimensionnalité
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)
X_train_reduced = pca.fit_transform(X_train)
X_test_reduced = pca.transform(X_test)

# Ré-entrainement du modèle
pipe.fit(X_train_reduced, y_train)

# Évaluation
y_pred = pipe.predict(X_test_reduced)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/home/yanis/miniconda3/envs/data_analyse_env/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/yanis/miniconda3/envs/data_analyse_env/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/yanis/miniconda3/envs/data_analyse_env/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, expli

KeyboardInterrupt: 

In [ ]:
predictions = pipe.predict(X_train)

In [ ]:
cross_val_score(pipe,X_train, y_train, scoring = 'f1_macro').mean()

In [ ]:
from sklearn.model_selection import cross_val_predict
pred_xg = cross_val_predict(pipe,X_train,y_train, cv=5,  verbose=0)
# on fait une prediction sur le y train 

In [ ]:
print(classification_report(y_train, predictions))

In [ ]:
predictions2 = model_xgb.predict(X_test)

In [ ]:
print(classification_report(y_test, predictions2))

In [ ]:
# Définir les hyperparamètres à évaluer
param_grid = { 
    'random_state':[0, 4294967295],
    'n_estimators': [200, 1000],
    'max_depth': [4, 6],
    'min_samples_split': [2000, 3000],
    'class_weight': ["balanced"],
    'criterion': ['gini'],
    'max_features': [1, 2, 3],
    'min_samples_leaf': [2, 3],
    'max_leaf_nodes': [None, 10, 20]
}

# Effectuer le GridSearch
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid,scoring='roc_auc', cv=5)
grid_search.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres
print(grid_search.best_params_)

KeyboardInterrupt: 

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(pipe,X_train,y_train, cv=5,  verbose=0)

# Calculer le score F1
f1_score(y_train, y_pred, average="macro")

In [ ]:
cross_val_score(pipe,X_train, y_train, scoring = 'f1_macro').mean()

In [ ]:
print(classification_report(y_train, y_pred))

In [ ]:
pipe[0].get_feature_names_out()

# Training

In [ ]:
from sklearn.metrics import f1_score
import numpy as np
def run_experiment(model):
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
#    if model est un grid_search:
    print('Best Hyperparameters: %s' % model.best_params_)
    print("######## R^2 : ")
    print("TRAIN :",r2_score(y_train, y_pred_train))
    print("TEST :",r2_score(y_test, y_pred_test))
    print("######## MAE : ")
    print("TRAIN :",mean_absolute_error(y_train, y_pred_train))
    print("TEST :",mean_absolute_error(y_test, y_pred_test))
    print("######## MSE : ")
    print("TRAIN :",mean_squared_error(y_train, y_pred_train))
    print("TEST :",mean_squared_error(y_test, y_pred_test))
    return model

# Justification Hyperparamètres - DecisionTreeRegressor

In [ ]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# define evaluation
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define search space
space = dict()

# gérer les valeurs abbérantes
space['decisiontree__criterion'] = ["squared_error", "absolute_error"] # absolute error traite mieux les valeurs abbérantes

# Gérer la profondeur de l'arbre
space['decisiontree__max_depth'] = np.arange(2,10,1) # plus c'est profond, plus ça over-fit (Un seul arbre, il faut un peu de profondeur pour ne pas avoir trop de biais)
# space['decisiontree__max_leaf_nodes'] = np.arange(10,100,5) # plus c'est grand, plus ça overfit 

# Gérer la taille des feuilles
# space['decisiontree__min_samples_split'] = np.arange(60,400,20) # plus c'est petit, plus ça overfit
space['decisiontree__min_samples_leaf'] = np.arange(30,150,10)  # plus c'est petit, plus ça overfit / on vise la loi des grands nombre autour de 50
# space['decisiontree__min_weight_fraction_leaf'] = np.arange(60,400,20) # plus c'est petit, plus ça overfit 

# Limiter les splits sans interets
space['decisiontree__min_impurity_decrease'] = np.linspace(0,1000,100) # plus c'est petit, plus ça overfit 
# space['decisiontree__ccp_alpha'] = np.linspace(0,1,100) # plus c'est petit, plus ça overfit 
# define search

random_search = RandomizedSearchCV(pipe, space, n_iter=1000, scoring='neg_mean_absolute_error', n_jobs=-1, cv=cv, random_state=1,verbose=2)



In [ ]:
run_experiment(random_search)

Best Hyperparameters: {'decisiontree__min_samples_split': 30, 'decisiontree__min_impurity_decrease': 20.2020202020202, 'decisiontree__max_depth': 7, 'decisiontree__criterion': 'absolute_error'}
######## R^2 : 
TRAIN : 0.8569505276410747
TEST : 0.8548320132085467
######## MAE : 
TRAIN : 1904.246198558466
TEST : 2120.079402294776
######## MAE : 
TRAIN : 20697412.74199028
TEST : 22345410.128570188


In [ ]:
grid_search_fit = run_experiment(grid_search)

Best Hyperparameters: {'decisiontree__criterion': 'absolute_error', 'decisiontree__max_depth': 7, 'decisiontree__min_impurity_decrease': 0.0, 'decisiontree__min_samples_split': 30}
######## R^2 : 
TRAIN : 0.8601105027263687
TEST : 0.8537802986897045
######## MAE : 
TRAIN : 1687.802174985033
TEST : 2047.5025273880594
######## MAE : 
TRAIN : 20240205.123421565
TEST : 22507298.38493529


# Feature Importance

In [ ]:
my_model.feature_importances_

array([0.36658101, 0.19407224, 0.02969388, 0.0013703 , 0.00146453,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.40681804])